In [2]:
#importing requirments
import pandas as pd
import ollama
import os
import json
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

creating embeddings

In [3]:

def create_embeddings(l):
    res = ollama.embed(
        model="qllama/bge-large-en-v1.5:q4_k_m",
        input=l
    )

    vector = res["embeddings"]
    return vector



In [3]:
# s = "my name is keyur"
# print(create_embeddings(s))

In [4]:
# l=["hi","by"]
# print(len(create_embeddings(l)))


In [ ]:

jsons = os.listdir("jsons")
my_dict = []
chunk_id =0
for json_file in jsons:
    with open(f"jsons/{json_file}") as f:
        content = json.load(f)
    print(f"creating embeddings for {json_file}")
    embeddings =create_embeddings([c['text'] for c in content["chunks"]])
    for i,chunk  in enumerate(content["chunks"]):
        chunk["chunk_id"] = chunk_id
        chunk["embeddings"] = embeddings[i]
        chunk_id += 1
        my_dict.append(chunk)
        
    
# print(my_dict)

creating embeddings for 01_Introduction to Programming .json
creating embeddings for 02_Some Amazing Python Programs .json
creating embeddings for 03_Modules and Pip .json
creating embeddings for 04_Our First Python Program .json
creating embeddings for 05_Comments.json


saving model

In [6]:

df = pd.DataFrame.from_records(my_dict)
df.to_pickle("data.pkl")


NameError: name 'my_dict' is not defined

pulling top matching chunks for incoming query and introspecting it

In [4]:
df = pd.read_pickle("data.pkl")
# df = pd.read_csv("embed_data.csv")    #we are not using csv cause it stored embeddings as list hence we didnt want that so were using pkl to store

In [5]:
df["embeddings"].dtype

dtype('O')

In [6]:
print(np.vstack(df['embeddings']).shape)

(1678, 1024)


In [61]:
incoming_query = input("enter a query :")
question_embedding = create_embeddings(incoming_query)[0]
# print(np.vstack(df['embeddings']).shape)
similarities  =cosine_similarity(np.vstack(df['embeddings']),[question_embedding]).flatten()
top_results = 5
max_idx = similarities.argsort()[::-1][0:top_results]

In [62]:
new_df  = df.loc[max_idx]
print(new_df[["number","title","text"]])    

     number     title                                          text
1218     05  Comments          I want to tell you what are comments
1407     05  Comments                  which help you in commenting
1318     05  Comments   written this thing that a comment is a part
1444     05  Comments                    in this way we can comment
1255     05  Comments          comments means you can keep any text


In [63]:
for index,item in new_df.iterrows():
    print(index,item["title"],item["number"],item["text"],item["Start"],item["end"])

1218 Comments 05  I want to tell you what are comments 188.0 190.0
1407 Comments 05  which help you in commenting 566.0 568.0
1318 Comments 05  written this thing that a comment is a part 388.0 390.0
1444 Comments 05  in this way we can comment 640.0 642.0
1255 Comments 05  comments means you can keep any text 262.0 264.0


In [96]:
prompt = f'''You are a helpful AI teaching assistant.

Your job is to answer the student's question using ONLY the information provided in the study material.
Present the answer in a structured, learner-friendly format with headings, bullet points, and clear guidance.
Answer as if you are guiding a student who is watching a course video.
Do NOT mention documents, chunks, embeddings, or internal processes.

Study Material:
================

{new_df[["number","title","text","Start","end"]]}
================

Student Question:
{incoming_query}

Write the answer in the following format:

 Topic:
 Video where it is taught
 Where exactly in the video (based on subtitles)
 
What the learner should do

Answer:

'''
with open("promp.txt","w") as f:
    f.write(prompt)

# getting response from llm
now were using llama2.3:3b which is small in size and faster


In [97]:
def inference(prompt):
    model =ollama.generate(
        model="llama3.2:3b",
        prompt=prompt,
        stream=False
    )
    response =  model["response"]
    return response


In [98]:
response = inference(prompt=prompt)

In [99]:
with open("output.txt","w") as f:
    f.write(response)